In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/xalan-2.6.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.6.0,org.apache.xalan.xsltc.compiler.FilterExpr,7,3,0,19,35,0,0,...,1.000000,1,0.922078,0.306122,2,9,31.000000,3,1.5714,1
1,xalan,2.6.0,org.apache.xpath.operations.Operation,12,2,14,22,23,0,17,...,1.000000,2,0.731707,0.218750,1,7,11.250000,5,1.5833,1
2,xalan,2.6.0,org.apache.xalan.xsltc.runtime.BasisLibrary,69,1,0,36,198,2330,20,...,0.303030,0,0.000000,0.102076,0,0,36.217391,49,4.2464,3
3,xalan,2.6.0,org.apache.xpath.objects.XString,49,3,2,42,100,1176,33,...,0.000000,1,0.606838,0.122222,3,21,14.081633,16,1.8571,3
4,xalan,2.6.0,org.apache.xalan.xsltc.StripFilter,1,1,0,11,1,0,11,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,xalan,2.6.0,org.apache.xalan.xsltc.compiler.VariableRef,2,4,0,17,25,1,1,...,0.000000,0,0.987013,0.625000,2,3,67.500000,9,4.5000,0
881,xalan,2.6.0,org.apache.xml.serializer.AttributesImplSerial...,6,2,0,6,20,0,6,...,0.666667,0,0.827586,0.416667,1,2,19.166667,3,2.0000,1
882,xalan,2.6.0,org.apache.xpath.objects.XNodeSetForDOM,7,5,0,5,18,0,1,...,0.000000,0,0.978417,0.285714,3,14,15.285714,1,0.4286,0
883,xalan,2.6.0,org.apache.xpath.compiler.PsuedoNames,1,1,0,0,2,0,0,...,0.000000,0,0.000000,1.000000,0,0,3.000000,0,0.0000,0


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
465,3,2,0,3,5,3,1,2,0,2.000000,24,0.000000,0,0.500000,0.555556,0,0,7.000000,2,1.3333
230,3,3,0,6,7,3,1,5,3,2.000000,24,0.000000,0,0.952381,0.555556,1,1,7.000000,1,0.6667
620,4,7,0,7,13,6,0,7,3,2.000000,81,0.000000,0,0.960000,0.500000,3,8,19.250000,1,0.7500
79,1,1,0,3,1,0,1,2,1,2.000000,1,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
688,13,1,0,2,27,0,1,1,13,0.694444,168,0.000000,1,0.000000,0.400000,0,0,11.692308,6,1.6923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1,1,0,0,1,0,0,0,1,2.000000,1,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
740,11,3,0,16,41,15,3,14,10,0.700000,228,1.000000,1,0.951456,0.233766,2,6,19.454545,4,1.2727
785,20,2,0,42,46,184,19,30,19,2.000000,630,0.000000,0,0.634615,0.230769,2,4,30.500000,16,2.2500
875,6,1,0,10,38,9,6,4,2,1.026087,388,0.652174,0,0.000000,0.300000,0,0,59.833333,10,2.8333


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
217,10,3,0,24,46,19,1,23,9,0.666667,250,1.000000,2,0.861538,0.237500,1,7,23.600000,4,1.6000
647,37,5,1,34,111,284,20,14,26,0.797980,2307,1.000000,2,0.830846,0.206019,2,33,60.756757,20,3.5135
307,7,1,0,12,37,21,5,7,7,2.000000,250,0.000000,0,0.000000,0.404762,0,0,34.714286,4,1.2857
396,14,1,20,22,21,53,20,2,12,0.794872,166,1.000000,0,0.000000,0.404762,0,0,10.428571,5,1.4286
162,9,2,0,16,32,28,0,16,7,0.625000,228,1.000000,2,0.461538,0.325000,1,1,24.000000,8,2.2222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,7,4,0,19,38,7,0,19,5,0.458333,272,1.000000,1,0.941176,0.285714,3,13,37.285714,12,3.2857
857,15,1,3,8,43,91,4,4,7,0.811688,833,0.818182,3,0.000000,0.267857,0,0,53.066667,14,3.6000
72,3,3,0,0,4,1,0,0,2,0.500000,1248,1.000000,0,0.974359,1.000000,0,0,414.666667,1,0.3333
258,3,3,0,0,4,1,0,0,2,0.500000,234,1.000000,0,0.974359,1.000000,0,0,76.666667,1,0.3333


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

465    0
230    0
620    0
79     0
688    1
      ..
109    0
740    2
785    0
875    2
32     6
Name: bug, Length: 708, dtype: int64

In [10]:
y_test

217    1
647    3
307    3
396    0
162    1
      ..
698    0
857    2
72     1
258    1
343    1
Name: bug, Length: 177, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[  3.        ,   2.        ,   0.        , ...,   7.        ,
          2.        ,   1.3333    ],
       [  3.        ,   3.        ,   0.        , ...,   7.        ,
          1.        ,   0.6667    ],
       [  4.        ,   7.        ,   0.        , ...,  19.25      ,
          1.        ,   0.75      ],
       ...,
       [ 20.        ,   2.        ,   0.        , ...,  30.5       ,
         16.        ,   2.25      ],
       [  6.        ,   1.        ,   0.        , ...,  59.83333333,
         10.        ,   2.8333    ],
       [  5.        ,   3.        ,   2.        , ..., 397.        ,
          2.        ,   0.8       ]])

In [13]:
x_test_np

array([[1.00000000e+01, 3.00000000e+00, 0.00000000e+00, ...,
        2.36000000e+01, 4.00000000e+00, 1.60000000e+00],
       [3.70000000e+01, 5.00000000e+00, 1.00000000e+00, ...,
        6.07567568e+01, 2.00000000e+01, 3.51350000e+00],
       [7.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        3.47142857e+01, 4.00000000e+00, 1.28570000e+00],
       ...,
       [3.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
        4.14666667e+02, 1.00000000e+00, 3.33300000e-01],
       [3.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
        7.66666667e+01, 1.00000000e+00, 3.33300000e-01],
       [9.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        7.23333333e+01, 1.70000000e+01, 3.88890000e+00]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[  3.        ,   2.        ,   0.        , ...,   7.        ,
           2.        ,   1.3333    ]],

       [[  3.        ,   3.        ,   0.        , ...,   7.        ,
           1.        ,   0.6667    ]],

       [[  4.        ,   7.        ,   0.        , ...,  19.25      ,
           1.        ,   0.75      ]],

       ...,

       [[ 20.        ,   2.        ,   0.        , ...,  30.5       ,
          16.        ,   2.25      ]],

       [[  6.        ,   1.        ,   0.        , ...,  59.83333333,
          10.        ,   2.8333    ]],

       [[  5.        ,   3.        ,   2.        , ..., 397.        ,
           2.        ,   0.8       ]]])

In [17]:
test_y

array([1, 3, 3, 0, 1, 0, 1, 5, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 4, 0, 0, 1, 1, 0, 0, 0,
       2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 2, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 1, 0, 0, 2, 1, 3, 0, 0, 0, 1, 0, 0,
       1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       2, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 2, 0, 1, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 2, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 2, 1, 1,
       1])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
23/23 [==============================] - 4s 12ms/step - loss: 1.3666 - mse: 1.3666 - mae: 0.7295 - root_mean_squared_error: 1.1690 - mean_squared_logarithmic_error: 0.3129
Epoch 2/100
23/23 [==============================] - 0s 5ms/step - loss: 0.9897 - mse: 0.9897 - mae: 0.6787 - root_mean_squared_error: 0.9948 - mean_squared_logarithmic_error: 0.2253
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8944 - mse: 0.8944 - mae: 0.6296 - root_mean_squared_error: 0.9457 - mean_squared_logarithmic_error: 0.1974
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8692 - mse: 0.8692 - mae: 0.6048 - root_mean_squared_error: 0.9323 - mean_squared_logarithmic_error: 0.1843
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8198 - mse: 0.8198 - mae: 0.6073 - root_mean_squared_error: 0.9055 - mean_squared_logarithmic_error: 0.1763
Epoch 6/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8255 - ms

23/23 [==============================] - 0s 5ms/step - loss: 0.5363 - mse: 0.5363 - mae: 0.4566 - root_mean_squared_error: 0.7323 - mean_squared_logarithmic_error: 0.1102
Epoch 45/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5712 - mse: 0.5712 - mae: 0.4420 - root_mean_squared_error: 0.7558 - mean_squared_logarithmic_error: 0.1137
Epoch 46/100
23/23 [==============================] - 0s 5ms/step - loss: 0.5533 - mse: 0.5533 - mae: 0.4567 - root_mean_squared_error: 0.7438 - mean_squared_logarithmic_error: 0.1133
Epoch 47/100
23/23 [==============================] - 0s 7ms/step - loss: 0.6073 - mse: 0.6073 - mae: 0.4794 - root_mean_squared_error: 0.7793 - mean_squared_logarithmic_error: 0.1233
Epoch 48/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5853 - mse: 0.5853 - mae: 0.4643 - root_mean_squared_error: 0.7650 - mean_squared_logarithmic_error: 0.1191
Epoch 49/100
23/23 [==============================] - 0s 5ms/step - loss: 0.5385 - mse: 0.538

23/23 [==============================] - 0s 5ms/step - loss: 0.4802 - mse: 0.4802 - mae: 0.3914 - root_mean_squared_error: 0.6930 - mean_squared_logarithmic_error: 0.0920
Epoch 89/100
23/23 [==============================] - 0s 5ms/step - loss: 0.4603 - mse: 0.4603 - mae: 0.3986 - root_mean_squared_error: 0.6784 - mean_squared_logarithmic_error: 0.0960
Epoch 90/100
23/23 [==============================] - 0s 6ms/step - loss: 0.5275 - mse: 0.5275 - mae: 0.4331 - root_mean_squared_error: 0.7263 - mean_squared_logarithmic_error: 0.1143
Epoch 91/100
23/23 [==============================] - 0s 7ms/step - loss: 0.4894 - mse: 0.4894 - mae: 0.4121 - root_mean_squared_error: 0.6996 - mean_squared_logarithmic_error: 0.1019
Epoch 92/100
23/23 [==============================] - 0s 5ms/step - loss: 0.4354 - mse: 0.4354 - mae: 0.3873 - root_mean_squared_error: 0.6598 - mean_squared_logarithmic_error: 0.0912
Epoch 93/100
23/23 [==============================] - 0s 5ms/step - loss: 0.4483 - mse: 0.448

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:23:00 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 1s 2ms/step - loss: 0.5394 - mse: 0.5394 - mae: 0.4796 - root_mean_squared_error: 0.7345 - mean_squared_logarithmic_error: 0.1644


[0.5394323468208313,
 0.5394323468208313,
 0.47964245080947876,
 0.7344605922698975,
 0.1643844097852707]